Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5930735 0.274840281 0.0043456064 0.0047371347 1000
 beta   0.9041204 0.041645831 0.0006584784 0.0006624367 1000
sigma   5.1049194 0.193773017 0.0030638204 0.0028243830 1000

Quantiles:
          2.5%         25.0%       50.0%       75.0%      97.5%   
alpha 154.05497500 154.41000000 154.5980000 154.781000 155.1290000
 beta   0.82252425   0.87594925   0.9034045   0.932612   0.9842076
sigma   4.73078250   4.97173750   5.1025500   5.230175   5.5100873



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.329 0.880803 5.10552; 154.823 0.899001 5.14815; … ; 154.72 0.944324 5.14454; 154.398 0.933555 5.08952]

Union{Missing, Float64}[154.379 1.00679 4.99746; 154.793 0.805284 5.04438; … ; 154.204 0.916521 5.12151; 154.609 0.900272 5.39311]

Union{Missing, Float64}[154.829 0.921788 5.37383; 154.907 0.848772 5.2303; … ; 154.648 0.879629 5.16733; 154.541 0.923374 4.99054]

Union{Missing, Float64}[154.257 0.954399 5.37343; 154.874 0.89111 4.99156; … ; 154.684 0.920282 4.51506; 154.746 0.924247 4.86233]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5930735 0.274840281 0.0043456064 0.0047371347 1000
 beta   0.9041204 0.041645831 0.0006584784 0.0006624367 1000
sigma   5.1049194 0.193773017 0.0030638204 0.0028243830 1000

Quantiles:
          2.5%         25.0%       50.0%       75.0%      97.5%   
alpha 154.05497500 154.41000000 154.5980000 154.781000 155.1290000
 beta   0.82252425   0.87594925   0.9034045   0.932612   0.9842076
sigma   4.73078250   4.97173750   5.1025500   5.230175   5.5100873



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*